In [ ]:
!pip install kaggle


In [ ]:
import os
import json
import pandas as pd
from zipfile import ZipFile


from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout,Embedding,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score



In [ ]:
file_path = '/content/IMDB Dataset.csv'
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            # You can add a condition here to stop at a certain line number if the error message gives a line number
            # For example, if the error was at line 7887, you could do:
            # if i > 7880:
            print(f"Line {i+1}: {line.strip()}")

except Exception as e:
    print(f"An error occurred while reading the file: {e}")

An error occurred while reading the file: [Errno 2] No such file or directory: '/content/IMDB Dataset.csv'


In [ ]:
df=pd.read_csv('/content/movie.csv', engine='python', on_bad_lines='skip')

In [ ]:
df.shape

(7134, 2)

In [ ]:
df.head(10)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
5,A terrible movie as everyone has said. What ma...,0
6,Finally watched this shocking movie last night...,1
7,I caught this film on AZN on cable. It sounded...,0
8,It may be the remake of 1987 Autumn's Tale aft...,1
9,My Super Ex Girlfriend turned out to be a plea...,1


In [ ]:
df.tail()

,text,label
7129,I should have known better..the clues were all...,0
7130,After having seen the Canadian/Icelandic/Briti...,0
7131,"In retrospect, the 1970s was a golden era for ...",1
7132,"For some unknown reason, 7 years ago, I watche...",0
7133,*** SPOILERS *** <br /><br />this movie always...,1


In [ ]:
df['label'].value_counts()

,count
label,
0,3630
1,3504


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['text'],df['label'],test_size=0.2,random_state=42)

In [ ]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)
x_trainpadas=tokenizer.texts_to_sequences(x_train)
x_testpadas=tokenizer.texts_to_sequences(x_test)

In [70]:
model=Sequential()
model.add(Embedding(5000,64,input_length=200))
model.add(LSTM(64,dropout=0.2))
model.add(Dense(1,activation='sigmoid'))


In [73]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [92]:
from tensorflow.keras.callbacks import EarlyStopping

# Pad the sequences to a fixed length
maxlen = 200  # You can adjust this based on your data
x_train_padded = pad_sequences(x_trainpadas, padding='post', maxlen=maxlen)
x_test_padded = pad_sequences(x_testpadas, padding='post', maxlen=maxlen)


early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train_padded, y_train, epochs=5, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 42s 127ms/step - accuracy: 0.9802 - loss: 0.0778 - val_accuracy: 0.8571 - val_loss: 0.4739
Epoch 2/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 84s 132ms/step - accuracy: 0.9854 - loss: 0.0616 - val_accuracy: 0.8578 - val_loss: 0.4485
Epoch 3/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 80s 126ms/step - accuracy: 0.9854 - loss: 0.0580 - val_accuracy: 0.8582 - val_loss: 0.5510
Epoch 4/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 81s 124ms/step - accuracy: 0.9894 - loss: 0.0440 - val_accuracy: 0.8597 - val_loss: 0.5376
Epoch 5/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 84s 130ms/step - accuracy: 0.9926 - loss: 0.0364 - val_accuracy: 0.8575 - val_loss: 0.5878


In [96]:
loss,accuracy=model.evaluate(x_test_padded,y_test)
print(f'loss:{loss},accuracy:{accuracy}')

105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8669 - loss: 0.5454
loss:0.569902777671814,accuracy:0.8614187240600586


In [98]:
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, padding='post', maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = 'positive' if prediction > 0.5 else 'negative'
    return sentiment

In [103]:
new_review="worth it"
sentiment=predict_sentiment(new_review)
print(f'The sentiment of the review is: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
The sentiment of the review is: positive


In [104]:
model.save("sentiment_model.h5")
